<a href="https://colab.research.google.com/github/hrcn81/commonplace/blob/main/surplus/mini_batchGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_diabetes

import random
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
X,y = load_diabetes(return_X_y = True)

In [3]:
print(X.shape)
print(y.shape)

(442, 10)
(442,)


In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [5]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [6]:
print(reg.coef_,'\n')
print(reg.intercept_)

[  37.90402135 -241.96436231  542.42875852  347.70384391 -931.48884588
  518.06227698  163.41998299  275.31790158  736.1988589    48.67065743] 

151.34560453985995


In [7]:
y_pred = reg.predict(X_test)
r2_score(y_test,y_pred)

0.4526027629719195

In [8]:
X_train.shape

(353, 10)

In [9]:
import random

class MBGDRegressor:

    def __init__(self,batch_size,learning_rate=0.01,epochs=100):

        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size

    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])

        for i in range(self.epochs):

            for j in range(int(X_train.shape[0]/self.batch_size)):

                idx = random.sample(range(X_train.shape[0]),self.batch_size)

                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                #print("Shape of y_hat",y_hat.shape)
                intercept_der = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.lr * coef_der)

        print(self.intercept_,self.coef_)

    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_

In [10]:
mbgdr = MBGDRegressor(batch_size = int(X_train.shape[0]/50),learning_rate = 0.01,epochs = 100)

In [11]:
mbgdr.fit(X_train,y_train)

149.5177096810033 [  53.70137306 -158.92333037  477.11425613  309.14748802  -48.74037347
 -100.02709489 -209.40281667  143.19242911  360.46615569  133.92056195]


In [12]:
y_pred = mbgdr.predict(X_test)

In [13]:
r2_score(y_test,y_pred)

0.4545078421795282

### Using sklearn

In [14]:
from sklearn.linear_model import SGDRegressor

In [15]:
sgd = SGDRegressor(learning_rate = 'constant',eta0 = 0.1)

In [16]:
batch_size = 50

for i in range(100):
    idx  = random.sample(range(X_train.shape[0]),batch_size)
    sgd.partial_fit(X_train[idx],y_train[idx])

In [17]:
sgd.coef_

array([  58.33215435, -122.97750193,  418.84581107,  271.11896434,
        -11.99834587,  -63.40715722, -186.5753841 ,  139.91376781,
        327.87745411,  132.0669174 ])

In [18]:
sgd.intercept_

array([150.68134617])

In [19]:
y_pred = sgd.predict(X_test)

In [20]:
r2_score(y_test,y_pred)

0.4570509799647937